# 역전파 학습 알고리즘의 구현과 실험

## 설치

In [ ]:
!pip install pillow # png 파일을 불러오기 위해 사용

## import

In [4]:
import random
import math
from PIL import Image

## 파라미터

In [5]:
input_size=16*16 # 16 x 16 픽셀
output_size = 7 # t, u, v, w, x, y, z 7개 
hidden_size = 1000
learning_cycle = 100 # 학습 사이클 횟수 
data_set = 10 # 데이터 셋의 개수

## 신경망 구현  
input_layer: 16 x 16 = 256개  
hidden_layer1: 1000개  
hidden_layer2: 1000개  
output_layer: 7개  

In [6]:
input_layer = [0]*input_size # input_layer를 만들어준다 
output_layer = [0]*output_size # output_layer를 만들어준다 
hidden_layer1 = [0]*hidden_size #hidden_layer1을 만들어준다
hidden_layer2 = [0]*hidden_size #hidden_layer2를 만들어준다


w1 = [[0 for col in range(input_size)]for row in range(hidden_size)] # 첫번째 가중치 (input_layer에서 hidden_layer로 가는 가중치)
w2 = [[0 for col in range(hidden_size)]for row in range(hidden_size)] # 두번째 가중치 (hidden_layer1에서 hidden_layer2로 가는 가중치)
w3 = [[0 for col in range(hidden_size)]for row in range(output_size)] # 세번째 가중치 (hidden_layer2에서 output_layer로 가는 가중치)

## 가중치 초기화 

In [7]:
# w1 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(input_size): # 열의 개수만큼 for문을 돌려준다 
        w1[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w2 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w2[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w3 초기화
for i in range(output_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w3[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)


## 행렬곱 함수

In [8]:
def matrixmult(A, B): 
    row_A = len(A)
    col_A = len(A[0])
    row_B = len(B) # B는 input 값이기 떄문에 항상 열 개수가 1이다
    
    if col_A != row_B:
        raise ValueError("행렬 곱셈이 불가능합니다")
    
    C=[0]*row_A # C의 크기는 A의 행 수 x B의 열 수 인데 B의 열 수는 1이라 제외 
    for i in range(row_A):
            for k in range(col_A):
                C[i] += A[i][k]*B[k] # 행렬곱 c[i][j] += A[i][k]*B[k][j](j는 B의 열 번호) 이지만 여기서는 B의 열 개수가 항상 1이므로 제외
    return C

## 전치 함수

In [9]:
def transeposed(a):
    t = list(zip(*a))
    return t

## 이미지 함수

In [10]:
def img_processing(alphabet,num):
    img_path = str('./image/learning/'+(chr(alphabet+ord('t')))+str(num+1)+'.png') 
    img = Image.open(img_path) # 이미지를 불러온다
    img = img.convert("L") # 이미지를 흑백으로 바꾼다 (0~255의 값을 가진다)
    img = list(img.getdata()) # 이미지 데이터를 list 형태로 바꾸어준다 
    
    for i in range(len(img)): # img가 0~1사이의 값을 가질 수 있도록 정규화 해준다 
        img[i] = img[i]/255 
    
    return img

## 순전파 함수

In [14]:
def FP(img): 
    #### input_layer ####
    for i in range(input_size):
        input_layer[i] = float(img[i]) # input_layer에 이미지를 넣어준다
    
    #### hidden_layer1 ####
    hidden_input1 = matrixmult(w1, input_layer) # 행렬곱을 해준다 (가중치 계산 input_layer -> hidden_layer1)
            
    for i in range(hidden_size):
        hidden_layer1[i] = hidden_input1[i] # hidden_layer1에 행렬곱한 결과를 넣어준다 
            
    hidden_output1 = [0]*hidden_size # hidden_layer1의 output을 초기화 해준다 
    for i in range(hidden_size): 
        hidden_output1[i] = max(0,hidden_layer1[i]) # ReLU연산을 해준다음 hidden_output1에 저장해준다 
    
    #### hidden_layer2 ####
    hidden_input2 = matrixmult(w2, hidden_output1) # 행렬곱을 해준다 (가중치 계산 hidden_layer1 -> hidden_layer2)
    
    for i in range(hidden_size):
        hidden_layer2[i]=hidden_input2[i] # hidden_layer2에 행렬곱한 결과를 넣어준다 
        
    hidden_output2 = [0]*hidden_size # hidden_layer2의 output을 초기화 해준다 
    for i in range(hidden_size):
        hidden_output2[i]=max(0,hidden_layer2[i]) # ReLU연산을 해준다음 hidden_output2에 저장해준다 
    
    #### output_layer ####
    output_input = matrixmult(w3, hidden_output2) # 행렬곱을 해준다 (가중치 계산 hidden_layer2 -> output_layer)
    
    for i in range(output_size):
        output_layer[i]=output_input[i] # output_layer에 행렬곱한 결과를 넣어준다 
        
    output_output = [0]*output_size # output_layer의 output을 초기화 해준다
    for i in range(output_size):
        output_output[i]=max(0,output_layer[i])
        
    return output_output
    
    
    

## 오차 역전파 함수

In [18]:
def BP(output, target):
    
    output_error=[0]*output_size
    for i in range(output_size):
        if i == target: # 만약 x가 target과 같다면 목표값을 1로 설정해준다
            output_error[i] = float(1 - output[i]) # 오차 = 목표값 - 결과값 
        else:    # target과 다르다면 목표값을 0으로 설정해준다 
            output_error[i] = float(0 - output[i]) # 오차 = 목표값 - 결과값
            
    hidden_error2 = matrixmult(transeposed(w3), output_error) # 가중치 행렬을 전치하게되면 역전파 행렬이 된다 
    hidden_error1 = matrixmult(transeposed(w2), hidden_error2)
    input_error = matrixmult(transeposed(w1), hidden_error1)
    
    return input_error, hidden_error1, hidden_error2, output_error
    

## 가중치 업데이트

In [ ]:
def update:
    

In [23]:
# 순전파 
for i in range(learning_cycle): # 학습 사이클 
    for j in range(output_size): # t~z 까지 
        for k in range(data_set): # 데이터셋의 개수만큼 반복
            # 이미지 가공 
            img = img_processing(j, k)
            output = FP(img)
            input_error, hidden_error1, hidden_error2, output_error = BP(output, j)
            
            
            
            
            
                
            
            
            
            

            

[103, 101, 99, 98, 98, 97, 96, 95, 94, 95, 96, 95, 95, 95, 94, 95, 105, 103, 101, 99, 98, 98, 96, 96, 95, 97, 97, 97, 96, 96, 96, 96, 107, 105, 103, 100, 100, 98, 100, 102, 66, 101, 100, 99, 99, 98, 97, 98, 112, 108, 106, 104, 103, 103, 101, 104, 65, 103, 101, 101, 101, 101, 100, 101, 115, 114, 112, 110, 108, 106, 105, 108, 66, 105, 104, 105, 104, 104, 104, 104, 117, 116, 114, 113, 113, 110, 109, 108, 68, 109, 107, 108, 108, 107, 106, 107, 120, 118, 118, 116, 114, 112, 112, 105, 76, 111, 111, 110, 109, 109, 109, 110, 122, 121, 119, 116, 114, 107, 94, 81, 61, 86, 84, 89, 112, 112, 113, 113, 124, 117, 88, 84, 85, 91, 97, 80, 95, 110, 111, 114, 115, 115, 115, 115, 127, 126, 124, 124, 122, 120, 121, 91, 105, 118, 118, 118, 118, 118, 118, 117, 128, 126, 125, 124, 123, 121, 122, 91, 109, 120, 120, 120, 120, 120, 120, 121, 129, 128, 126, 125, 125, 123, 124, 95, 108, 123, 123, 123, 123, 123, 122, 122, 130, 128, 127, 125, 126, 124, 125, 99, 108, 123, 124, 124, 124, 124, 124, 123, 131, 130, 128,

[151, 151, 151, 151, 151, 150, 151, 149, 129, 120, 141, 150, 149, 149, 150, 150, 151, 150, 150, 150, 150, 149, 149, 146, 100, 145, 149, 148, 149, 149, 150, 151, 151, 150, 149, 149, 149, 148, 148, 136, 101, 150, 149, 148, 149, 149, 150, 150, 151, 150, 150, 149, 148, 148, 147, 125, 116, 150, 149, 148, 148, 149, 148, 149, 151, 150, 149, 148, 148, 148, 146, 108, 128, 149, 147, 148, 149, 148, 148, 149, 151, 150, 150, 150, 148, 148, 144, 96, 146, 147, 148, 148, 149, 149, 150, 149, 149, 142, 152, 150, 149, 148, 138, 90, 149, 147, 149, 150, 152, 152, 148, 149, 146, 110, 88, 87, 98, 125, 116, 99, 141, 129, 117, 99, 88, 112, 150, 149, 152, 152, 148, 147, 129, 110, 83, 70, 81, 92, 107, 130, 149, 152, 150, 149, 151, 150, 151, 151, 149, 149, 109, 133, 150, 152, 152, 150, 151, 151, 150, 149, 151, 150, 150, 150, 150, 146, 100, 143, 151, 151, 150, 150, 150, 150, 150, 150, 150, 150, 149, 150, 149, 145, 101, 152, 151, 151, 151, 150, 150, 150, 150, 150, 151, 150, 150, 150, 151, 143, 100, 151, 151, 151, 1

[157, 157, 156, 157, 156, 157, 156, 155, 155, 155, 156, 156, 156, 155, 155, 156, 157, 157, 157, 155, 156, 156, 156, 156, 155, 153, 154, 156, 156, 156, 155, 156, 157, 156, 156, 152, 152, 158, 156, 156, 156, 148, 106, 157, 155, 155, 156, 156, 157, 156, 155, 151, 114, 145, 156, 155, 154, 144, 102, 156, 155, 155, 156, 155, 157, 157, 155, 144, 125, 158, 156, 156, 152, 129, 105, 156, 155, 155, 155, 156, 157, 157, 156, 140, 127, 156, 156, 156, 150, 114, 118, 155, 156, 156, 155, 156, 158, 157, 156, 131, 131, 156, 155, 156, 147, 98, 136, 155, 155, 155, 156, 157, 158, 158, 156, 127, 134, 155, 155, 152, 128, 99, 150, 156, 156, 155, 155, 155, 158, 157, 156, 130, 130, 154, 153, 147, 120, 108, 158, 157, 155, 155, 156, 156, 158, 158, 158, 144, 101, 154, 153, 130, 142, 105, 157, 157, 158, 156, 156, 155, 157, 157, 157, 157, 104, 100, 117, 140, 154, 108, 159, 157, 156, 156, 156, 156, 156, 157, 157, 156, 158, 155, 150, 156, 156, 133, 153, 156, 156, 155, 156, 156, 157, 158, 157, 158, 157, 158, 156, 156, 1

[147, 147, 146, 146, 146, 146, 146, 146, 147, 146, 146, 146, 147, 146, 145, 145, 147, 147, 147, 146, 146, 146, 146, 145, 146, 148, 149, 146, 146, 146, 146, 146, 147, 147, 147, 147, 145, 145, 145, 145, 147, 125, 120, 146, 145, 146, 146, 145, 146, 146, 146, 121, 146, 146, 146, 145, 145, 96, 123, 145, 145, 145, 146, 145, 146, 145, 138, 90, 147, 146, 146, 145, 144, 90, 121, 145, 145, 144, 145, 145, 146, 146, 141, 92, 147, 145, 146, 146, 143, 90, 121, 145, 144, 144, 144, 144, 146, 146, 144, 90, 148, 146, 145, 146, 138, 92, 127, 145, 144, 144, 144, 145, 147, 148, 145, 90, 147, 145, 147, 145, 118, 95, 137, 144, 145, 145, 145, 145, 147, 148, 149, 92, 146, 148, 147, 146, 102, 97, 141, 144, 145, 144, 144, 145, 147, 148, 147, 105, 133, 146, 147, 139, 105, 95, 141, 145, 145, 145, 144, 145, 148, 147, 148, 130, 104, 149, 148, 120, 134, 108, 128, 145, 145, 144, 144, 145, 147, 146, 147, 142, 92, 139, 138, 102, 146, 132, 108, 147, 145, 145, 145, 145, 147, 146, 147, 147, 135, 74, 79, 139, 147, 144, 140,

[153, 153, 153, 153, 152, 152, 153, 153, 152, 153, 153, 152, 151, 152, 151, 152, 152, 153, 152, 152, 152, 152, 152, 153, 152, 153, 153, 152, 152, 152, 152, 152, 152, 152, 153, 152, 151, 152, 153, 152, 152, 152, 153, 152, 152, 152, 151, 152, 152, 153, 153, 153, 153, 153, 152, 152, 152, 153, 153, 154, 134, 152, 151, 151, 153, 153, 153, 153, 153, 152, 152, 152, 151, 152, 151, 135, 113, 151, 151, 152, 152, 154, 153, 150, 140, 151, 151, 152, 151, 152, 150, 109, 141, 151, 151, 151, 151, 151, 152, 153, 103, 122, 153, 152, 150, 150, 145, 106, 153, 151, 151, 150, 151, 151, 152, 151, 150, 79, 152, 151, 152, 150, 116, 128, 151, 150, 150, 151, 151, 152, 152, 152, 153, 129, 109, 151, 151, 151, 98, 150, 150, 150, 151, 151, 152, 152, 151, 152, 151, 150, 72, 152, 151, 142, 111, 152, 150, 151, 151, 152, 151, 151, 151, 152, 152, 154, 125, 108, 153, 109, 139, 151, 151, 152, 151, 151, 151, 151, 151, 152, 152, 153, 153, 81, 103, 101, 152, 151, 150, 151, 150, 150, 151, 152, 151, 151, 152, 152, 153, 152, 97,

KeyboardInterrupt: 

In [29]:
for i in range(len(img)):
        img[i] = img[i]/255

In [30]:
img

[0.48627450980392156,
 0.49019607843137253,
 0.49411764705882355,
 0.4980392156862745,
 0.5019607843137255,
 0.5058823529411764,
 0.5058823529411764,
 0.5058823529411764,
 0.5058823529411764,
 0.5098039215686274,
 0.5098039215686274,
 0.5098039215686274,
 0.5137254901960784,
 0.5137254901960784,
 0.5137254901960784,
 0.5176470588235295,
 0.4980392156862745,
 0.5019607843137255,
 0.5058823529411764,
 0.5098039215686274,
 0.5137254901960784,
 0.5176470588235295,
 0.5176470588235295,
 0.5176470588235295,
 0.5215686274509804,
 0.5215686274509804,
 0.5176470588235295,
 0.5176470588235295,
 0.5254901960784314,
 0.5215686274509804,
 0.5215686274509804,
 0.5254901960784314,
 0.5098039215686274,
 0.5137254901960784,
 0.5176470588235295,
 0.5254901960784314,
 0.5294117647058824,
 0.5294117647058824,
 0.5294117647058824,
 0.5294117647058824,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333,
 0.5450980392156862,
 0.5450980392156862,
 0.5450980392156862,
 0.5490196